## Score en Kaggle: 0.83358

In [2]:
import numpy as np
import scipy.stats as stats

In [3]:
import lightgbm as lgbm

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [5]:
from sklearn.model_selection import RandomizedSearchCV

In [6]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [9]:
processed = pd.read_csv('features_04.csv')

In [10]:
processed = processed.set_index(processed.columns[0])
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,1362,False,False,True,False,False,True,False,True,False
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,1155,False,True,True,True,True,True,True,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,12924,True,True,True,True,False,True,False,True,False


In [11]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [12]:
predict_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
00091926,viewed product,iPhone 6S,Computer,0,34,372,448,28,0,16.000000,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,10,1,66,10.000000,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,206,10,5,20.600000,...,1155,False,True,True,True,True,True,True,True,False
000e4d9e,viewed product,Samsung Galaxy S6 Flat,Computer,0,13,339,411,15,1,27.400000,...,335,False,True,True,True,False,True,True,True,False
000e619d,viewed product,Motorola Moto G2 3G Dual,Computer,0,5,28,68,3,13,22.666667,...,1155,True,True,True,True,False,True,True,True,False


In [13]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,...,1362,False,False,True,False,False,True,False,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,...,12924,True,True,True,True,False,True,False,True,False
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,...,76,True,True,True,False,False,True,False,True,False
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,...,8873,True,True,True,True,True,True,True,True,False
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,...,588,False,False,True,False,True,True,True,False,False


## Mean encoding

In [15]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [16]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,...,1362,False,False,True,False,False,True,False,True,False
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,...,12924,True,True,True,True,False,True,False,True,False
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,...,76,True,True,True,False,False,True,False,True,False
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,...,8873,True,True,True,True,True,True,True,True,False
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,...,588,False,False,True,False,True,True,True,False,False


In [37]:
params = {
    'num_leaves':stats.randint(60,120),
    'min_data_in_leaf':stats.randint(100,1000),
    'max_bin':stats.randint(70,300),
    'learning_rate':stats.uniform(0.01,1-0.01),
    'n_iter':stats.randint(150,500)
}
grid = RandomizedSearchCV(lgbm.LGBMClassifier(),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=0,
                          n_iter=50)

In [38]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [39]:
grid.fit(xtr,ytr)

C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_iter` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_iter` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_iter` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_iter` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_iter` in pa

KeyboardInterrupt: 

In [27]:
grid.score(X=xte,y=yte)

0.8704919809276115

In [28]:
grid.best_params_

{'learning_rate': 0.05173305648482968,
 'max_bin': 156,
 'min_data_in_leaf': 488,
 'num_iterations': 162,
 'num_leaves': 91}

## Prediccion

In [29]:
predictor = grid.best_estimator_

In [30]:
predictor.feature_importances_

array([ 34, 323,   8,  60,  90, 230, 379, 160, 397, 284,  32,   0, 196,
         0, 255, 158, 168,  20,   4,  31, 130,  54,   0, 117, 139,  34,
         7,  85,  59,   0,  24,  15,   0,   0])

In [31]:
predictor.fit(encoded_training,training_labels)

C:\Users\javif\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.05173305648482968,
        max_bin=156, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=488, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_iterations=162, num_leaves=91,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [32]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [33]:
predict_features['label']=prediction_submit

In [34]:
submit = predict.set_index('person')

In [35]:
submit['label'] = predict_features['label']

In [36]:
submit.to_csv('submit.csv')